# Parsing Ironic Conductor logs from Scale testing
Note - the code is very ad-hoc and loads full Ironic Conductor log in memory to parse and analyze it.

In [ ]:
import datetime
import re
import pandas

In [ ]:
with open('/home/pshchelo/Downloads/log') as f:
    rawstr = f.readlines()

In [ ]:
dt_re_str = "(?P<timestamp>\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3})"
uuid_re_str = "(?P<uuid>[0-9a-f]{8}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{12})"
tsformat = "%Y-%m-%d %H:%M:%S.%f"

In [ ]:
# regexps for matching events
start_re = re.compile("{ts}.*DEBUG.*RPC do_node_deploy called for node {uuid}".format(ts=dt_re_str, uuid=uuid_re_str))
done_re = re.compile("{ts}.*INFO.*Deployment to node {uuid} done".format(ts=dt_re_str, uuid=uuid_re_str))
ans_start_re = re.compile("{ts}.*DEBUG ironic_staging_drivers\.ansible\.deploy.*Call back from {uuid}\.".format(ts=dt_re_str, uuid=uuid_re_str))
ipa_start_re = re.compile("{ts}.*INFO.* Initial lookup for node {uuid} succeeded".format(ts=dt_re_str, uuid=uuid_re_str))
fuel_start_re = re.compile("{ts}.*DEBUG.*RPC vendor_passthru called for node {uuid}".format(ts=dt_re_str, uuid=uuid_re_str))

In [ ]:
# time slots of tests
slot_format = "%Y-%m-%d %H:%M"
fuel_time_slot = (datetime.datetime.strptime("2016-06-27 16:43", slot_format), datetime.datetime.strptime("2016-06-27 16:59", slot_format))
ipa_time_slot = (datetime.datetime.strptime("2016-06-27 17:13", slot_format), datetime.datetime.strptime("2016-06-27 17:27", slot_format))
ansible_time_slot = (datetime.datetime.strptime("2016-06-27 17:35", slot_format), datetime.datetime.strptime("2016-06-27 17:46", slot_format))
ipa_tor_time_slot = (datetime.datetime.strptime("2016-06-27 17:53", slot_format), datetime.datetime.strptime("2016-06-27 18:02", slot_format))
ansible_tor_time_slot = (datetime.datetime.strptime("2016-06-27 18:10", slot_format), datetime.datetime.strptime("2016-06-27 18:18", slot_format))

In [ ]:
def parse_events(raw_data, driver_name, driver_matcher):
    nodes = {}
    for s in raw_data:
        m = start_re.match(s)
        if m:
            uuid = m.groupdict()['uuid']
            ts = m.groupdict()['timestamp']
            if uuid not in nodes:
                nodes[uuid] = []
            nodes[uuid].append(dict(time=datetime.datetime.strptime(ts, tsformat), action='start', driver=None, match=m))
            continue
        m2 = driver_matcher.match(s)
        if m2:
            uuid = m2.groupdict()['uuid']
            ts = m2.groupdict()['timestamp']
            if uuid not in nodes:
                nodes[uuid] = []
            nodes[uuid].append(dict(time=datetime.datetime.strptime(ts, tsformat), action='deploy', driver=driver_name, match=m2))
            continue
        m3 = done_re.match(s)
        if m3:
            uuid = m3.groupdict()['uuid']
            ts = m3.groupdict()['timestamp']
            if uuid not in nodes:
                nodes[uuid] = []
            nodes[uuid].append(dict(time=datetime.datetime.strptime(ts, tsformat), action='done', driver=None, match=m3))
    return nodes

In [ ]:
def filter_complete_cycles(events, driver):
    deploys = {}
    for node, val in events.items():
        for i, item in enumerate(val):
            if item['driver'] == driver:
                if node not in deploys:
                    deploys[node] = []
                if val[i-1]['action'] == 'start' and val[i+1]['action'] == 'done':
                    deploy = dict(start=val[i-1]['time'],
                                  deploy=val[i]['time'],
                                  done=val[i+1]['time'])
                    deploy['delta_all'] = deploy['done'] - deploy['start']
                    deploy['delta_boot'] = deploy['deploy'] - deploy['start']
                    deploy['delta_deploy'] = deploy['done'] - deploy['deploy']
                    deploys[node].append(deploy)
    return deploys

In [ ]:
def test_run(all_deploys, time_slot):
    runs = []
    for node, deploys in all_deploys.items():
        for deploy in deploys:
            if (deploy['start'] >= time_slot[0] and deploy['done'] <= time_slot[1] ):
                runs.append(deploy)
    return runs

In [ ]:
def get_run_stats(runs, metric):
    series = pandas.Series([i[metric] for i in runs])
    print(series.describe())

In [ ]:
ansible_deploys = filter_complete_cycles(parse_events(rawstr, 'ansible', ans_start_re), 'ansible')

In [ ]:
ansible_http_runs = test_run(ansible_deploys, ansible_time_slot)
get_run_stats(ansible_http_runs, 'delta_all')
get_run_stats(ansible_http_runs, 'delta_deploy')

In [ ]:
ansible_tor_runs = test_run(ansible_deploys, ansible_tor_time_slot)
get_run_stats(ansible_tor_runs, 'delta_all')
get_run_stats(ansible_tor_runs, 'delta_deploy')

In [ ]:
ipa_deploys = filter_complete_cycles(parse_events(rawstr, 'ipa', ipa_start_re), 'ipa')

In [ ]:
ipa_http_runs = test_run(ipa_deploys, ipa_time_slot)
get_run_stats(ipa_http_runs, 'delta_all')
get_run_stats(ipa_http_runs, 'delta_deploy')

In [ ]:
ipa_tor_runs = test_run(ipa_deploys, ipa_tor_time_slot)
get_run_stats(ipa_tor_runs, 'delta_all')
get_run_stats(ipa_tor_runs, 'delta_deploy')

In [ ]:
fuel_deploys = filter_complete_cycles(parse_events(rawstr, 'fuel', fuel_start_re), 'fuel')

In [ ]:
fuel_runs = test_run(fuel_deploys, fuel_time_slot)
get_run_stats(fuel_runs, 'delta_all')
get_run_stats(fuel_runs, 'delta_deploy')